### A. Librerias

In [1]:
import os
import time
import json
import requests

import pandas as pd

from utils import sheets
from bs4 import BeautifulSoup

### B. Funciones

In [2]:
def get_files(path, extension):
    dir_name  = os.path.join(os.getcwd(), path)
    files     = os.listdir(dir_name)
    
    paths    = []
    for file in files:
        if extension in file:
            paths.append(os.path.join(path,file))

    return paths

def clean_files(path, extension='.json'):
    try:
        files = get_files(path, extension)
        for file in files:
            os.remove(file)
        print('Se eliminaron los archivos de la carpeta')
    except:
        pass

def generar_url_google_maps(latitud, longitud):
    url = f"https://www.google.com/maps?q={latitud},{longitud}"
    return url

def generar_url_street_view(latitud, longitud):
    url = f"https://www.google.com/maps?q=&layer=c&cbll={latitud},{longitud}"
    return url

### C. Ejecución - Buscar departamentos por distritos

In [3]:
# 1. Listar los distritos y sus urls
distritos_urls = [
                  ['all_nexo', 'https://nexoinmobiliario.pe/busqueda/venta-de-departamentos-o-oficinas-o-lotes-o-casas']
                ]

In [4]:
# 2. Extraer los datos de cada distrito
for distrito, url in distritos_urls:
    # 1. Obtener la URL por request
    response = requests.get(url)
    print(response.status_code, '-> ', distrito)

    if response.status_code == 200:
        # 2. Parsear el HTML
        html = response.text

        # 3. Guardar el HTML en un archivo
        # with open(os.path.join(os.getcwd(),'originals', distrito + '.html'), 'w') as outfile:
        with open(os.path.join(os.getcwd(),'originals', distrito + '.html'), 'w', encoding='utf-8') as outfile:
            outfile.write(html)

        # 4. Tiempo de espera
        time.sleep(2)

200 ->  all_nexo


In [5]:
# 3. Listar los proyectos y sus urls
urls_list = []
for distrito, x in distritos_urls:
    # A. Leer el html
    # with open(os.path.join(os.getcwd(),'originals', distrito+ '.html'), 'r') as infile:
    with open(os.path.join(os.getcwd(),'originals', distrito + '.html'), 'r', encoding='utf-8', errors='ignore') as infile:
        lines = infile.readlines()

    # B. Buscar la línea que contiene la palabra 'url'
    cadena = ''
    for i in range(len(lines)-1,0,-1):
        if '"url":' in lines[i]:
            cadena = lines[i]
            break
    
    # C. Buscar las urls
    posiciones_url = []
    posicion = cadena.find('"url":')

    while posicion != -1:
        posiciones_url.append(posicion)
        posicion = cadena.find('"url":', posicion + 1)
    
    # D. Buscar los project_id
    posiciones_ids = []
    posicion = cadena.find('"project_id":')

    while posicion != -1:
        posiciones_ids.append(posicion)
        posicion = cadena.find('"project_id":', posicion + 1)

    # E. Buscar links
    for a,b in zip(posiciones_url, posiciones_ids):
        url = ''.join(list(cadena)[a + 7 :b - 2])
        url = url.replace('\x5C','')  
        url = str(url)

        base = 'https://nexoinmobiliario.pe/proyecto/venta-de-departamento-' + url.split('/')[-1]
        urls_list.append([distrito,base])

len(urls_list)

656

In [6]:
# 4. Extraer los datos de cada proyecto
i = 0
modelos_list = []
dataframes_list = []
for distrito, url in urls_list:
    # A. Obtener la URL por request
    response = requests.get(url)
    # print(response.status_code,':' ,url)

    if response.status_code == 200:
        # Parsea el contenido HTML de la página
        soup = BeautifulSoup(response.text, 'html.parser')

        #### A. Datos generales
        # Extrae el texto de <h1>Eres Fase 2</h1>
        try:
            proyecto = soup.find('h1').get_text(strip=True)
        except:
            proyecto = '-'
        # print(f'Proyecto: {proyecto}')

        # Extrae el texto de <p class="Project-header-address">Av. República de Panamá 4077</p>
        try:
            direccion = soup.find('p', class_='Project-header-address').get_text(strip=True)
        except:
            direccion = '-'
        # print(f'Dirección: {direccion}')

        # Extrae el texto de <p class="Project-header-address"> - Surquillo</p>
        try:
            distrito_nexo = soup.find('p', class_='Project-header-address').find_next_sibling('p').get_text(strip=True).replace('-', '').strip()
        except:
            distrito_nexo = '-'
        # print(f'Distrito: {distrito}')

        # Extrae el precio desde <p class="Project-header-price"><span class="title-price">Precio desde</span> <strong> S/. 319,118</strong></p>
        try:
            precio_desde = soup.find('p', class_='Project-header-price').strong.get_text(strip=True)
        except:
            precio_desde = '-'
        # print(f'Precio desde: {precio_desde}')

        # Extrae el valor de referencia <p class="title-price">(<span>valor referencia </span><strong>$ 84,940</strong>)</p>
        try:
            valor_referencia = soup.find('p', class_='title-price').strong.get_text(strip=True)
        except:
            valor_referencia = '-'
        # print(f'Valor de referencia: {valor_referencia}')

        # Listar las áreas comunes
        try:
            areas_lista = soup.find('ul', class_='Project-areas-list')
            elementos_lista = areas_lista.find_all('li')
            areas_comunes = ', '.join(elemento.get_text(strip=True) for elemento in elementos_lista)
        except:
            areas_comunes = '-'

        # Encuentra el campo de entrada para la latitud
        try:
            input_latitud = soup.find('input', id='latitude')
            latitud = input_latitud['value']
        except:
            latitud = '-'
        # print(f'Latitud: {latitud}')

        try:
            input_longitud = soup.find('input', id='longitude')
            longitud = input_longitud['value']
        except:
            longitud = '-'
        # print(f'Longitud: {longitud}')

        try:
            url_google_maps = generar_url_google_maps(latitud, longitud)
        except:
            url_google_maps = '-'


        # print('')
        # print('Datos de la inmobiliaria')
        #### B. Datos de la inmobiliaria
        try:
            # Encuentra el div con la clase "bx-data-project box-st"
            div_informacion_proyecto = soup.find('div', class_='bx-data-project box-st')

            if div_informacion_proyecto:
                # Encuentra todas las filas de la tabla dentro del div
                filas_tabla = div_informacion_proyecto.find('table').find_all('tr')

                # Itera sobre las filas de la tabla e imprime la información
                for fila in filas_tabla:
                    columnas = fila.find_all('td')
                    if len(columnas) == 2:
                        try:
                            etiqueta = columnas[0].strong.get_text(strip=True)
                            valor = columnas[1].get_text(strip=True)
                            
                            if etiqueta == 'Tipo de inmueble':
                                tipo_inmueble = valor
                            elif etiqueta == 'Área total':
                                area_total = valor
                            elif etiqueta == 'Dormitorios':
                                dormitorios = valor
                            elif etiqueta == 'Etapa del proyecto':
                                etapa_proyecto = valor
                            elif etiqueta == 'Fecha de entrega':
                                fecha_entrega = valor
                            elif etiqueta == 'Financiamiento':
                                financiamiento = valor
                            elif etiqueta == 'Inmobiliaria Nombre':
                                inmobiliaria_nombre = valor
                            
                            # print(f'{etiqueta} -> {valor}')

                        except:
                            pass

                # También puedes extraer información específica usando el método find para elementos específicos
                # Por ejemplo, para obtener la información de la inmobiliaria
                inmobiliaria_nombre = div_informacion_proyecto.find('tr', class_='Project-inmobiliaria').find('h2').get_text(strip=True)
                # print(f'Inmobiliaria Nombre: {inmobiliaria_nombre}')
        except:
            tipo_inmueble = '-'
            area_total = '-'
            dormitorios = '-'
            etapa_proyecto = '-'
            fecha_entrega = '-'
            financiamiento = '-'
            inmobiliaria_nombre = '-'
            # print('No se encontró información de la inmobiliaria')

        #### C. Modelos disponibles
        try:
            modelos_disponibles = soup.find_all('div', class_='Project-available-model')
            modelos_list.append([  proyecto,modelos_disponibles, distrito, distrito_nexo, tipo_inmueble
                                 , etapa_proyecto, fecha_entrega, financiamiento
                                 , inmobiliaria_nombre, areas_comunes
                                 , url_google_maps, url])
            # print('')
            # print('Modelos disponibles')
            # print(f'Cantidad de modelos: {len(modelos_disponibles)}')
        except:
            pass

        #### D. Unir los datos en un diccionario
        proyecto_data = {
            'Proyecto': proyecto,
            'Dirección': direccion,
            'Distrito': distrito,
            'Distrito Nexoinmobiliario': distrito_nexo,
            'Precio Desde': precio_desde,
            'Valor de Referencia': valor_referencia,
            'Latitud': latitud,
            'Longitud': longitud,
            'Tipo de Inmueble': tipo_inmueble,
            'Área Total': area_total,
            'Dormitorios': dormitorios,
            'Etapa del Proyecto': etapa_proyecto,
            'Fecha de Entrega': fecha_entrega,
            'Financiamiento': financiamiento,
            'Inmobiliaria Nombre': inmobiliaria_nombre,
            'Áreas Comunes': areas_comunes,
            'URL Nexoinmobiliario': url,
            'URL Google Maps': url_google_maps

        }

        # Convierte el diccionario en un DataFrame
        df = pd.DataFrame([proyecto_data])
        dataframes_list.append(df)

        # Incrementa el contador
        print(i, '-> ', proyecto , '-> ', url)
        i += 1
        
        # Tiempo de espera
        time.sleep(1.21)

0 ->  Edificio Orizzon 2 ->  https://nexoinmobiliario.pe/proyecto/venta-de-departamento-2607-edificio-orizzon-2
1 ->  Mozz ->  https://nexoinmobiliario.pe/proyecto/venta-de-departamento-2219-mozz
2 ->  Concepto Urban Park (Etapa II) ->  https://nexoinmobiliario.pe/proyecto/venta-de-departamento-2727-concepto-urban-park-etapa-ii
3 ->  Mar de Plata II ->  https://nexoinmobiliario.pe/proyecto/venta-de-departamento-1693-mar-de-plata-ii
4 ->  Ceibos del Chipe ->  https://nexoinmobiliario.pe/proyecto/venta-de-departamento-2303-ceibos-del-chipe
5 ->  BENAVIDES 1130 ->  https://nexoinmobiliario.pe/proyecto/venta-de-departamento-2307-benavides-1130
6 ->  PROYECTO PIEROLA ->  https://nexoinmobiliario.pe/proyecto/venta-de-departamento-3192-proyecto-pierola
7 ->  Nesta Terra ->  https://nexoinmobiliario.pe/proyecto/venta-de-departamento-2191-nesta-terra
8 ->  MARISCAL CASTILLA ->  https://nexoinmobiliario.pe/proyecto/venta-de-departamento-2722-mariscal-castilla
9 ->  Invent Barranco ->  https://ne

In [7]:
# 1. Concatenar los DataFrames
df_proyectos = pd.concat(dataframes_list)

# 2. Eliminar las columnas de Latitud y Longitud
df_proyectos = df_proyectos.drop(columns=['Latitud', 'Longitud'])
df_proyectos.head(3)

,Proyecto,Dirección,Distrito,Distrito Nexoinmobiliario,Precio Desde,Valor de Referencia,Tipo de Inmueble,Área Total,Dormitorios,Etapa del Proyecto,Fecha de Entrega,Financiamiento,Inmobiliaria Nombre,Áreas Comunes,URL Nexoinmobiliario,URL Google Maps
0,Edificio Orizzon 2,Jirón Daniel Carrión 110,all_nexo,Oyague Magdalena Del Mar,"S/. 412,421","$ 110,332",Departamento,56.69 a 156.63 m2,2 a 3,En construcción,"31 de Julio, 2024",BBVA,VITAIN,"Área de juegos para niños, Lobby, Sala Bar, Zo...",https://nexoinmobiliario.pe/proyecto/venta-de-...,"https://www.google.com/maps?q=-12.0949213,-77...."
0,Mozz,Odriozola 126,all_nexo,San Isidro,"$ 118,151","S/. 441,648",Departamento,45.00 a 60.12 m2,1 a 2,Entrega inmediata,"31 de Diciembre, 2022",Banco BCP,VALICO,"Areas verdes, Gimnasio, Sala de usos Múltiples",https://nexoinmobiliario.pe/proyecto/venta-de-...,"https://www.google.com/maps?q=-12.0981572,-77...."
0,Concepto Urban Park (Etapa II),Av. Petit Thouars 1140,all_nexo,Santa Beatriz Cercado de lima,"S/. 290,709","$ 77,771",Departamento,40.65 a 86.38 m2,1 a 3,En construcción,"30 de Junio, 2024",Banco Scotiabank,IMAGINA,"Gimnasio, Jardin interior, Lobby, Piscina, Sal...",https://nexoinmobiliario.pe/proyecto/venta-de-...,"https://www.google.com/maps?q=-12.0758939,-77...."


### D. Ejecución - Buscar detalle de departamentos

In [8]:
data_modelos = []
for   proyecto,modelos_disponibles, distrito, distrito_nexo, tipo_inmueble, etapa_proyecto, fecha_entrega, financiamiento \
    , inmobiliaria_nombre, areas_comunes, url_google_maps, url in modelos_list:
    for modelo in modelos_disponibles:
        try:
            # Extrae información del enlace de imagen
            #imagen_enlace = modelo.find('a', class_='popup-link')['data-links'].strip()
            #print(f'Enlace de la imagen: {imagen_enlace}')

            # Extrae información del modelo
            nombre_modelo = modelo.find('span', class_='name_tipology').get_text(strip=True).strip()
            #print(f'Nombre del modelo: {nombre_modelo}')

            # Extrae información del número de pisos
            num_pisos = modelo.find('span', class_='num_pisos').get_text(strip=True).strip()
            #print(f'Número de pisos: {num_pisos}')

            # Extrae información del dormitorio, área y precio
            dormitorio = modelo.find('span', class_='bedroom').get_text(strip=True).strip()
            area = modelo.find('span', class_='area').get_text(strip=True).strip()
            precio = modelo.find('span', class_='price').get_text(strip=True).strip()

            #print(f'Dormitorio: {dormitorio}')
            #print(f'Área: {area}')
            #print(f'Precio desde: {precio}')

            # También puedes extraer información del enlace de cotización
            #enlace_cotizacion = modelo.find('a', class_='Btn-ver--red')['href'].strip()
            #print(f'Enlace de cotización: {enlace_cotizacion}')

            # Agregar datos del modelo a la lista
            data_modelos.append({
                'Proyecto': proyecto,
                'Distrito': distrito,
                'Distrito Nexoinmobiliario': distrito_nexo,
                'Tipo de Inmueble': tipo_inmueble,
                'Etapa del Proyecto': etapa_proyecto,
                'Fecha de Entrega': fecha_entrega,
                'Nombre del modelo': nombre_modelo,
                'Número de pisos': num_pisos,
                'Dormitorio': dormitorio,
                'Área': area,
                'Precio desde': precio,
                'Financiamiento': financiamiento,
                'Inmobiliaria Nombre': inmobiliaria_nombre,
                'Áreas Comunes': areas_comunes,
                'URL Nexoinmobiliario': url,
                'URL Google Maps': url_google_maps
            })

            #print("\n---\n")  # Separador entre modelos
        except:
            pass

df_modelos = pd.DataFrame(data_modelos)
df_modelos.head(3)

,Proyecto,Distrito,Distrito Nexoinmobiliario,Tipo de Inmueble,Etapa del Proyecto,Fecha de Entrega,Nombre del modelo,Número de pisos,Dormitorio,Área,Precio desde,Financiamiento,Inmobiliaria Nombre,Áreas Comunes,URL Nexoinmobiliario,URL Google Maps
0,Edificio Orizzon 2,all_nexo,Oyague Magdalena Del Mar,Departamento,En construcción,"31 de Julio, 2024",104,Piso: 1,3,94.50 ...,"S/ 562,815",BBVA,VITAIN,"Área de juegos para niños, Lobby, Sala Bar, Zo...",https://nexoinmobiliario.pe/proyecto/venta-de-...,"https://www.google.com/maps?q=-12.0949213,-77...."
1,Edificio Orizzon 2,all_nexo,Oyague Magdalena Del Mar,Departamento,En construcción,"31 de Julio, 2024",DPTO TIPO 1-A,Pisos: 4 y 6,3,76.38 ...,"S/ 591,945",BBVA,VITAIN,"Área de juegos para niños, Lobby, Sala Bar, Zo...",https://nexoinmobiliario.pe/proyecto/venta-de-...,"https://www.google.com/maps?q=-12.0949213,-77...."
2,Edificio Orizzon 2,all_nexo,Oyague Magdalena Del Mar,Departamento,En construcción,"31 de Julio, 2024",DPTO TIPO 3-A,Pisos: 2 y 8,3,76.02 ...,"S/ 589,155",BBVA,VITAIN,"Área de juegos para niños, Lobby, Sala Bar, Zo...",https://nexoinmobiliario.pe/proyecto/venta-de-...,"https://www.google.com/maps?q=-12.0949213,-77...."


### E. Ejecución - Guardar Parquet/Excel

In [9]:
# Guardar df_proyectos y df_modelos en un parquet
df_proyectos.to_parquet(os.path.join(os.getcwd(),'outputs','df_proyectos.parquet'))
df_modelos.to_parquet(os.path.join(os.getcwd(),'outputs','df_modelos.parquet'))

# Guardar df_proyectos y df_modelos en excel
df_proyectos.to_excel(os.path.join(os.getcwd(),'outputs','proyectos.xlsx'), index=False)
df_modelos.to_excel(os.path.join(os.getcwd(),'outputs','modelos.xlsx'), index=False)

### F. Dar formato a df_proyectos

In [10]:
import os
import pandas as pd

df_proyectos = pd.read_parquet(os.path.join(os.getcwd(),'outputs','df_proyectos.parquet'))
df_proyectos.head(1)

,Proyecto,Dirección,Distrito,Distrito Nexoinmobiliario,Precio Desde,Valor de Referencia,Tipo de Inmueble,Área Total,Dormitorios,Etapa del Proyecto,Fecha de Entrega,Financiamiento,Inmobiliaria Nombre,Áreas Comunes,URL Nexoinmobiliario,URL Google Maps
0,Edificio Orizzon 2,Jirón Daniel Carrión 110,all_nexo,Oyague Magdalena Del Mar,"S/. 412,421","$ 110,332",Departamento,56.69 a 156.63 m2,2 a 3,En construcción,"31 de Julio, 2024",BBVA,VITAIN,"Área de juegos para niños, Lobby, Sala Bar, Zo...",https://nexoinmobiliario.pe/proyecto/venta-de-...,"https://www.google.com/maps?q=-12.0949213,-77...."


In [11]:
df_distritos = pd.read_excel(os.path.join(os.getcwd(),'outputs','distritos.xlsx'))
df_distritos.head(2)

,Distrito Nexoinmobiliario,Distrito,Departamento/Provincia,Flg Departamento
0,CERRO AZUL,CERRO AZUL,LIMA PROVINCIAS,LIMA PROVINCIAS
1,MALA,MALA,LIMA PROVINCIAS,LIMA PROVINCIAS


In [12]:
def formato_fecha(mes_str):
    meses = {
        'Enero': '01',
        'Febrero': '02',
        'Marzo': '03',
        'Abril': '04',
        'Mayo': '05',
        'Junio': '06',
        'Julio': '07',
        'Agosto': '08',
        'Septiembre': '09',
        'Octubre': '10',
        'Noviembre': '11',
        'Diciembre': '12'
    }

    mes_split = mes_str.split(' ')

    dia = mes_split[0]
    mes = meses.get(mes_split[2].replace(',',''))
    anio = mes_split[3]

    mes_format = f'{anio}-{mes}-{dia}'
    if mes_format:
        return mes_format
    else:
        return None

In [13]:
def get_monto_moneda(valor_str_1, valor_str_2, key):
    valor_str_1 = str(valor_str_1)
    valor_str_2 = str(valor_str_2)

    if key in valor_str_1:
        return valor_str_1.replace(key, '').replace(',','').replace('.','').strip()
    elif key in valor_str_2:
        return valor_str_2.replace(key, '').replace(',','').replace('.','').strip()

In [14]:
# 1. Dar formato a las columnas de fecha y castear a date 'yyyy-mm-dd'
df_proyectos_format = df_proyectos.copy()
df_proyectos_format['Fecha de Entrega'] = df_proyectos_format['Fecha de Entrega'].apply(formato_fecha)
df_proyectos_format['Fecha de Entrega'] = pd.to_datetime(df_proyectos_format['Fecha de Entrega'])

# 2. Hacer un upper a las columas que son string
df_proyectos_format['Proyecto'] = df_proyectos_format['Proyecto'].str.upper()
df_proyectos_format['Distrito'] = df_proyectos_format['Distrito'].str.upper()
df_proyectos_format['Distrito Nexoinmobiliario'] = df_proyectos_format['Distrito Nexoinmobiliario'].str.upper()
df_proyectos_format['Dirección'] = df_proyectos_format['Dirección'].str.upper()
df_proyectos_format['Financiamiento'] = df_proyectos_format['Financiamiento'].str.upper()
df_proyectos_format['Inmobiliaria Nombre'] = df_proyectos_format['Inmobiliaria Nombre'].str.upper()
df_proyectos_format['Precio Desde'] = df_proyectos_format['Precio Desde'].str.upper()
df_proyectos_format['Valor de Referencia'] = df_proyectos_format['Valor de Referencia'].str.upper()

# 3. Reemplazar los valores de 'Precio Desde' y 'Valor de Referencia' por 0
df_proyectos_format['Precio Desde'] = df_proyectos_format['Precio Desde'].replace('-', 'S/. 0')
df_proyectos_format['Valor de Referencia'] = df_proyectos_format['Valor de Referencia'].replace('-', 'S/. 0')
 
# 5. Crear el campo 'Precio Desde (USD)'  y 'Precio Desde (PEN)'
df_proyectos_format['Precio Desde (USD)'] = df_proyectos_format.apply(lambda row: get_monto_moneda(row['Precio Desde'], row['Valor de Referencia'], '$'), axis=1)
df_proyectos_format['Precio Desde (PEN)'] = df_proyectos_format.apply(lambda row: get_monto_moneda(row['Precio Desde'], row['Valor de Referencia'], 'S/.'), axis=1)

# 6. Castear a float las columnas de precio
df_proyectos_format['Precio Desde (USD)'] = df_proyectos_format['Precio Desde (USD)'].astype(float)
df_proyectos_format['Precio Desde (PEN)'] = df_proyectos_format['Precio Desde (PEN)'].astype(float)

# 7. Reemplazar los valores de NaN por el tipo de cambio
df_proyectos_format['Precio Desde (USD)'] = df_proyectos_format['Precio Desde (USD)'].fillna(df_proyectos_format['Precio Desde (PEN)'] / 3.75)
df_proyectos_format['Precio Desde (PEN)'] = df_proyectos_format['Precio Desde (PEN)'].fillna(df_proyectos_format['Precio Desde (USD)'] * 3.75)

# 8. Castear Precio Desde (USD) y Precio Desde (PEN) a int
df_proyectos_format['Precio Desde (USD)'] = df_proyectos_format['Precio Desde (USD)'].astype(int)
df_proyectos_format['Precio Desde (PEN)'] = df_proyectos_format['Precio Desde (PEN)'].astype(int)

# 9. Area Total Min y Area Total Max
df_proyectos_format['Area Min (m2)'] = df_proyectos_format['Área Total'].apply(lambda x: x.split(' a ')[0].strip())
df_proyectos_format['Area Max (m2)'] = df_proyectos_format['Área Total'].apply(lambda x: x.split(' a ')[1].split('m2')[0].strip())

# 10. Reemplazar '.' por ',' en las columnas de Área Total	, Area Min (m2) y Area Max (m2)
df_proyectos_format['Área Total'] = df_proyectos_format['Área Total'].str.replace('.',',')
df_proyectos_format['Area Min (m2)'] = df_proyectos_format['Area Min (m2)'].str.replace('.',',')
df_proyectos_format['Area Max (m2)'] = df_proyectos_format['Area Max (m2)'].str.replace('.',',')

# 11. Eliminar la columna 'Distrito', luego merge con df_distritos por 'Distrito Nexoinmobiliario'
df_proyectos_format = df_proyectos_format.drop(columns=['Distrito'])
df_proyectos_format = df_proyectos_format.merge(df_distritos, how='left', on='Distrito Nexoinmobiliario')
df_proyectos_format['Distrito'] = df_proyectos_format['Distrito'].fillna('-')
df_proyectos_format['Flg Departamento'] = df_proyectos_format['Flg Departamento'].fillna('FUERA DE LIMA')

# 12. Reordenar las columnas
df_proyectos_format = df_proyectos_format[['Distrito', 'Proyecto', 'Flg Departamento', 'Distrito Nexoinmobiliario', 'Dirección',  'Tipo de Inmueble', 'Precio Desde','Área Total', 'Precio Desde (PEN)', 'Precio Desde (USD)',
                                           'Area Min (m2)', 'Area Max (m2)',
                                           'Dormitorios', 'Etapa del Proyecto', 'Fecha de Entrega', 'Financiamiento',
                                           'Inmobiliaria Nombre', 'Áreas Comunes', 'URL Nexoinmobiliario',
                                           'URL Google Maps'
                                           ]]

# Ordenar precio desde menor a mayor
df_proyectos_format.head(1)

,Distrito,Proyecto,Flg Departamento,Distrito Nexoinmobiliario,Dirección,Tipo de Inmueble,Precio Desde,Área Total,Precio Desde (PEN),Precio Desde (USD),Area Min (m2),Area Max (m2),Dormitorios,Etapa del Proyecto,Fecha de Entrega,Financiamiento,Inmobiliaria Nombre,Áreas Comunes,URL Nexoinmobiliario,URL Google Maps
0,MAGDALENA,EDIFICIO ORIZZON 2,LIMA,OYAGUE MAGDALENA DEL MAR,JIRÓN DANIEL CARRIÓN 110,Departamento,"S/. 412,421","56,69 a 156,63 m2",412421,110332,"56,69","156,63",2 a 3,En construcción,2024-07-31,BBVA,VITAIN,"Área de juegos para niños, Lobby, Sala Bar, Zo...",https://nexoinmobiliario.pe/proyecto/venta-de-...,"https://www.google.com/maps?q=-12.0949213,-77...."


In [15]:
df_proyectos_format.dtypes

Distrito                             object
Proyecto                             object
Flg Departamento                     object
Distrito Nexoinmobiliario            object
Dirección                            object
Tipo de Inmueble                     object
Precio Desde                         object
Área Total                           object
Precio Desde (PEN)                    int32
Precio Desde (USD)                    int32
Area Min (m2)                        object
Area Max (m2)                        object
Dormitorios                          object
Etapa del Proyecto                   object
Fecha de Entrega             datetime64[ns]
Financiamiento                       object
Inmobiliaria Nombre                  object
Áreas Comunes                        object
URL Nexoinmobiliario                 object
URL Google Maps                      object
dtype: object

### G. Dar formato a df_modelos

In [16]:
df_modelos = pd.read_parquet('outputs/df_modelos.parquet')

df_modelos.head(3)

,Proyecto,Distrito,Distrito Nexoinmobiliario,Tipo de Inmueble,Etapa del Proyecto,Fecha de Entrega,Nombre del modelo,Número de pisos,Dormitorio,Área,Precio desde,Financiamiento,Inmobiliaria Nombre,Áreas Comunes,URL Nexoinmobiliario,URL Google Maps
0,Edificio Orizzon 2,all_nexo,Oyague Magdalena Del Mar,Departamento,En construcción,"31 de Julio, 2024",104,Piso: 1,3,94.50 ...,"S/ 562,815",BBVA,VITAIN,"Área de juegos para niños, Lobby, Sala Bar, Zo...",https://nexoinmobiliario.pe/proyecto/venta-de-...,"https://www.google.com/maps?q=-12.0949213,-77...."
1,Edificio Orizzon 2,all_nexo,Oyague Magdalena Del Mar,Departamento,En construcción,"31 de Julio, 2024",DPTO TIPO 1-A,Pisos: 4 y 6,3,76.38 ...,"S/ 591,945",BBVA,VITAIN,"Área de juegos para niños, Lobby, Sala Bar, Zo...",https://nexoinmobiliario.pe/proyecto/venta-de-...,"https://www.google.com/maps?q=-12.0949213,-77...."
2,Edificio Orizzon 2,all_nexo,Oyague Magdalena Del Mar,Departamento,En construcción,"31 de Julio, 2024",DPTO TIPO 3-A,Pisos: 2 y 8,3,76.02 ...,"S/ 589,155",BBVA,VITAIN,"Área de juegos para niños, Lobby, Sala Bar, Zo...",https://nexoinmobiliario.pe/proyecto/venta-de-...,"https://www.google.com/maps?q=-12.0949213,-77...."


In [17]:
def get_monto_pen(precio_str, tipo_cambio = 3.75):
    precio_str = str(precio_str)
    if 'S/' in precio_str:
        precio_int = precio_str.replace('S/', '').replace(',','').replace('.','').strip()
        return int(precio_int)

    elif '$' in precio_str:
        precio_int = precio_str.replace('$', '').replace(',','').replace('.','').strip()
        return int(int(precio_int) * tipo_cambio)

In [18]:
def get_monto_usd(precio_str, tipo_cambio = 3.75):
    precio_str = str(precio_str)
    if '$' in precio_str:
        precio_int = precio_str.replace('$', '').replace(',','').replace('.','').strip()
        return int(precio_int)

    elif 'S/' in precio_str:
        precio_int = precio_str.replace('S/', '').replace(',','').replace('.','').strip()
        return int(int(precio_int) / tipo_cambio)

In [19]:
# 1. Dar formato a las columnas de fecha y castear a date 'yyyy-mm-dd'
df_modelos_format = df_modelos.copy()

# 2. Hacer un upper a las columas que son string
df_modelos_format['Proyecto'] = df_modelos_format['Proyecto'].str.upper()
df_modelos_format['Distrito'] = df_modelos_format['Distrito'].str.upper()
df_modelos_format['Distrito Nexoinmobiliario'] = df_modelos_format['Distrito Nexoinmobiliario'].str.upper()
df_modelos_format['Nombre del modelo'] = df_modelos_format['Nombre del modelo'].str.upper()
df_modelos_format['Financiamiento'] = df_modelos_format['Financiamiento'].str.upper()
df_modelos_format['Inmobiliaria Nombre'] = df_modelos_format['Inmobiliaria Nombre'].str.upper()


# 3. Formatear la columna 'Fecha de Entrega'
df_modelos_format['Fecha de Entrega'] = df_modelos_format['Fecha de Entrega'].apply(formato_fecha)
df_modelos_format['Fecha de Entrega'] = pd.to_datetime(df_modelos_format['Fecha de Entrega'])

# 3. Hacer .split(' ')[0] a la columna 'Área'
df_modelos_format['Área'] = df_modelos_format['Área'].apply(lambda x: x.split(' ')[0])

# 4. Crear el campo 'Precio Desde (USD)'  y 'Precio Desde (PEN)'
df_modelos_format['Precio Desde (USD)'] = df_modelos_format.apply(lambda row: get_monto_usd(row['Precio desde'], 3.75), axis=1)
df_modelos_format['Precio Desde (PEN)'] = df_modelos_format.apply(lambda row: get_monto_pen(row['Precio desde'], 3.75), axis=1)

# 5. Castear a float las columnas de precio
df_modelos_format['Precio Desde (USD)'] = df_modelos_format['Precio Desde (USD)'].astype(int)
df_modelos_format['Precio Desde (PEN)'] = df_modelos_format['Precio Desde (PEN)'].astype(int)

# 8. Calcular el precio por m2, el numero resultante debe tener los decimales separados por ,

df_modelos_format['Precio por m2 (USD)'] = df_modelos_format.apply(lambda row: round(float(row['Precio Desde (USD)'] )/ float(row['Área']),2), axis=1)
df_modelos_format['Precio por m2 (USD)'] = df_modelos_format['Precio por m2 (USD)'].astype(str)
df_modelos_format['Precio por m2 (USD)'] = df_modelos_format['Precio por m2 (USD)'].str.replace('.',',')

# 6. Reemplazar '.' por ',' en la columna 'Área'
df_modelos_format['Área'] = df_modelos_format['Área'].str.replace('.',',')

# 7. Renombrar columna 'Número de pisos':'Piso del departamento', 'Área':'Área (m2)', 'Dormitorio':'Dormitorios'
df_modelos_format = df_modelos_format.rename(columns={'Número de pisos':'Piso del departamento', 'Área':'Área (m2)', 'Dormitorio':'Dormitorios'})

# 8. Eliminar la columna 'Distrito', luego merge con df_distritos por 'Distrito Nexoinmobiliario'
df_modelos_format = df_modelos_format.drop(columns=['Distrito'])
df_modelos_format = df_modelos_format.merge(df_distritos, how='left', on='Distrito Nexoinmobiliario')
df_modelos_format['Distrito'] = df_modelos_format['Distrito'].fillna('-')
df_modelos_format['Flg Departamento'] = df_modelos_format['Flg Departamento'].fillna('FUERA DE LIMA')

# 9. Reordenar las columnas
df_modelos_format = df_modelos_format[[
                                        'Distrito', 'Proyecto', 'Flg Departamento', 'Distrito Nexoinmobiliario', 'Tipo de Inmueble', 
                                        'Etapa del Proyecto', 'Fecha de Entrega', 'Nombre del modelo', 'Piso del departamento',
                                        'Dormitorios', 'Área (m2)', 'Precio desde','Precio por m2 (USD)',
                                        'Precio Desde (USD)', 'Precio Desde (PEN)',
                                        'Financiamiento', 'Áreas Comunes', 'Inmobiliaria Nombre','URL Nexoinmobiliario', 'URL Google Maps' 
                                       ]]

df_modelos_format.head(1)

,Distrito,Proyecto,Flg Departamento,Distrito Nexoinmobiliario,Tipo de Inmueble,Etapa del Proyecto,Fecha de Entrega,Nombre del modelo,Piso del departamento,Dormitorios,Área (m2),Precio desde,Precio por m2 (USD),Precio Desde (USD),Precio Desde (PEN),Financiamiento,Áreas Comunes,Inmobiliaria Nombre,URL Nexoinmobiliario,URL Google Maps
0,MAGDALENA,EDIFICIO ORIZZON 2,LIMA,OYAGUE MAGDALENA DEL MAR,Departamento,En construcción,2024-07-31,104,Piso: 1,3,"94,50","S/ 562,815","1588,19",150084,562815,BBVA,"Área de juegos para niños, Lobby, Sala Bar, Zo...",VITAIN,https://nexoinmobiliario.pe/proyecto/venta-de-...,"https://www.google.com/maps?q=-12.0949213,-77...."


In [20]:
df_modelos_format.dtypes

Distrito                             object
Proyecto                             object
Flg Departamento                     object
Distrito Nexoinmobiliario            object
Tipo de Inmueble                     object
Etapa del Proyecto                   object
Fecha de Entrega             datetime64[ns]
Nombre del modelo                    object
Piso del departamento                object
Dormitorios                          object
Área (m2)                            object
Precio desde                         object
Precio por m2 (USD)                  object
Precio Desde (USD)                    int32
Precio Desde (PEN)                    int32
Financiamiento                       object
Áreas Comunes                        object
Inmobiliaria Nombre                  object
URL Nexoinmobiliario                 object
URL Google Maps                      object
dtype: object

### G. Guardar en google sheets 
drive-sheets-service@molten-topic-396921.iam.gserviceaccount.com  
https://docs.google.com/spreadsheets/d/1PDphDA-IHCnuykZoDWjzkl_f0AxOsw0bUsIbpE3weZg/edit#gid=0


In [21]:
from utils import sheets

# 1. Crear flujo de lectura y escritura
sh = sheets.autentificar(os.path.join(os.getcwd(),'utils', 'keys_sheets.json'), '1PDphDA-IHCnuykZoDWjzkl_f0AxOsw0bUsIbpE3weZg','id')

# 2. Guardar df_proyectos_format en la hoja 'Proyectos'
sheets.dataframe_to_sheet(sh, df_proyectos_format, 'Proyectos')

# 3. Guardar df_modelos_format en la hoja 'Detalles'
sheets.dataframe_to_sheet(sh, df_modelos_format, 'Departamentos')

In [22]:
print('Se guardaron los datos en Google Sheets')

Se guardaron los datos en Google Sheets
